## pipeline

In [1]:
import networkx as nx
import numpy as np
import torch

from func.run_pipeline_super_vox import get_outlayer_of_a_3d_shape, get_crop_by_pixel_val
from func.ultis import load_obj

from func.graph_learning import SuperVoxToNxGraph, VoxelGraphDataset

Using backend: pytorch


In [10]:
HMS_data_dict = load_obj("dataset_info/HMS_dataset_info")
HMS_data_dict_test = HMS_data_dict["test"]
print("Test cases: "+str(HMS_data_dict_test.keys()))
case = "135"
print("for test case "+str(case)+" : "+str(HMS_data_dict_test[case]))

# you may load the image using another path
raw_img=np.load(HMS_data_dict_test[case]["raw"]).astype(float)
hand_seg=np.load(HMS_data_dict_test[case]["ins"]).astype(float)

# np.save('seg_foreground_super_voxel_by_ws_graph.npy', seg_foreground_super_voxel_by_ws)

Test cases: dict_keys(['135', '120', '65', '90'])
for test case 135 : {'raw': 'data/CellSeg_dataset/HMS_processed/raw/135.npy', 'background': 'data/CellSeg_dataset/HMS_processed/segmentation/135/135_background_3d_mask.npy', 'edge': 'data/CellSeg_dataset/HMS_processed/segmentation/135/135_edge_3d_mask.npy', 'edge_foreground': 'data/CellSeg_dataset/HMS_processed/segmentation/135/135_edge_foreground_3d_mask.npy', 'edge_background': 'data/CellSeg_dataset/HMS_processed/segmentation/135/135_edge_background_3d_mask.npy', 'boundary': 'data/CellSeg_dataset/HMS_processed/segmentation/135/135_boundary_3d_mask.npy', 'foreground': 'data/CellSeg_dataset/HMS_processed/segmentation/135/135_foreground_3d_mask.npy', 'ins': 'data/CellSeg_dataset/HMS_processed/segmentation/135/135_ins.npy'}


In [11]:
seg_foreground_super_voxel_by_ws = np.load('seg_foreground_super_voxel_by_ws_graph.npy')


In [12]:
super_vox_to_graph = SuperVoxToNxGraph()

In [13]:
graph = super_vox_to_graph.get_nx_graph_from_ws_with_gt(seg_foreground_super_voxel_by_ws, hand_seg)

getting neighbor pairs
adding ground truth
adding neighbor ids
calculate edges
build networkx graph


In [60]:
from dgl.data import DGLDataset
import dgl



In [61]:
dataset = VoxelGraphDataset([graph])

g = dataset[0]

In [62]:
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Node features
{'feat': tensor([[16.],
        [11.],
        [ 2.],
        ...,
        [ 4.],
        [ 2.],
        [28.]]), 'label': tensor([1, 1, 1,  ..., 1, 1, 1]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ...,  True,  True,  True])}
Edge features
{'weight': tensor([69., 69., 35.,  ...,  0.,  0.,  0.], dtype=torch.float64)}


# TODO probably should normalize features!!!!

In [67]:
from func.graph_models import GCN
import torch.nn.functional as F

def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc))

model = GCN(1, 16, num_classes=2)
train(g, model)


In epoch 0, loss: 0.451, val acc: 0.911 (best 0.911)
In epoch 5, loss: 0.378, val acc: 0.911 (best 0.911)
In epoch 10, loss: 0.365, val acc: 0.911 (best 0.911)
In epoch 15, loss: 0.344, val acc: 0.911 (best 0.911)
In epoch 20, loss: 0.330, val acc: 0.911 (best 0.911)
In epoch 25, loss: 0.319, val acc: 0.911 (best 0.911)
In epoch 30, loss: 0.311, val acc: 0.911 (best 0.911)
In epoch 35, loss: 0.304, val acc: 0.911 (best 0.911)
In epoch 40, loss: 0.299, val acc: 0.911 (best 0.911)
In epoch 45, loss: 0.297, val acc: 0.911 (best 0.911)
In epoch 50, loss: 0.295, val acc: 0.911 (best 0.911)
In epoch 55, loss: 0.294, val acc: 0.911 (best 0.911)
In epoch 60, loss: 0.294, val acc: 0.911 (best 0.911)
In epoch 65, loss: 0.294, val acc: 0.911 (best 0.911)
In epoch 70, loss: 0.294, val acc: 0.911 (best 0.911)
In epoch 75, loss: 0.294, val acc: 0.911 (best 0.911)
In epoch 80, loss: 0.294, val acc: 0.911 (best 0.911)
In epoch 85, loss: 0.294, val acc: 0.911 (best 0.911)
In epoch 90, loss: 0.294, val 

# Train on GPU
g = g.to('cuda')
model = GCN(1, 16, dataset.num_classes).to('cuda')
train(g, model)

In [58]:
g

Graph(num_nodes=1860, num_edges=7174,
      ndata_schemes={'feat': Scheme(shape=(1,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.float32), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)})

In [59]:

g.ndata['feat'].shape

torch.Size([1860, 1])

In [46]:
import dgl.data

dataset_cora = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000


RuntimeError: Bool type is not supported by dlpack

In [43]:
dataset_cora

NameError: name 'dataset_cora' is not defined

In [44]:

!pip install dgl -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html
